## Let's get started using sockets!

Import the socket library

In [ ]:
import socket

The Python sockets library allows us to find out what port a service uses.

In [2]:
socket.getservbyname('ssh')

22

You can also do a reverse lookup, finding what service uses a given port.

In [3]:
socket.getservbyport(80)

'http'

The sockets library also provides tools for finding out information about hosts. For example, you can find out about the hostname and IP address of the machine you are currently using.

In [4]:
socket.gethostname()

'TREE'

In [5]:
socket.gethostbyname(socket.gethostname())

'192.168.3.37'

You can also find out about machines that are located elsewhere, assuming you know their hostname. For example:

In [6]:
socket.gethostbyname('google.com')

'216.58.193.78'

In [7]:
socket.gethostbyname('uw.edu')

'128.95.155.198'

In [9]:
socket.gethostbyname('cutecatvideos.net')

'50.97.112.130'

The gethostbyname_ex method of the socket library provides more information about the machines we are exploring:

In [12]:
socket.gethostbyname_ex('cutecatvideos.net')

('cutecatvideos.net', [], ['50.97.112.130'])

In [13]:
socket.gethostbyname_ex('google.com')

('google.com', [], ['216.58.193.78'])

To create a socket, you use the socket method of the socket library. It takes up to three optional positional arguments (here we use none to get the default behavior):

In [14]:
foo = socket.socket()

In [15]:
foo

<socket.socket fd=1204, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>

A socket has some properties that are immediately important to us. These include the family, type and protocol of the socket:

In [16]:
foo.family

<AddressFamily.AF_INET: 2>

In [17]:
foo.type

<SocketKind.SOCK_STREAM: 1>

In [18]:
foo.proto

0

You might notice that the values for these properties are integers. In fact, these integers are constants defined in the socket library.

Let’s define a method in place to help us see these constants. It will take a single argument, the shared prefix for a defined set of constants:

In [19]:
def get_constants(prefix):
    """mapping of socket module constants to their names"""
    return {getattr(socket, n): n
        for n in dir(socket)
        if n.startswith(prefix)
   }

## Socket Families

Think back a moment to our discussion of the Internet layer of the TCP/IP stack. There were a couple of different types of IP addresses:

IPv4 (‘192.168.1.100’)
IPv6 (‘2001:0db8:85a3:0042:0000:8a2e:0370:7334’)
The family of a socket corresponds to the addressing system it uses for connecting.

Families defined in the socket library are prefixed by AF_:

In [20]:
families = get_constants('AF_')

In [21]:
families

{<AddressFamily.AF_UNSPEC: 0>: 'AF_UNSPEC',
 <AddressFamily.AF_INET: 2>: 'AF_INET',
 <AddressFamily.AF_IPX: 6>: 'AF_IPX',
 <AddressFamily.AF_SNA: 11>: 'AF_SNA',
 12: 'AF_DECnet',
 <AddressFamily.AF_APPLETALK: 16>: 'AF_APPLETALK',
 <AddressFamily.AF_INET6: 23>: 'AF_INET6',
 <AddressFamily.AF_IRDA: 26>: 'AF_IRDA'}

Your results may vary

Of all of these, the ones we care most about are 2 (IPv4) and 30 (IPv6).

When you are on a machine with an operating system that is Unix-like, you will find another generally useful socket family: AF_UNIX, or Unix Domain Sockets. Sockets in this family:

connect processes on the same machine
are generally a bit slower than IPC connnections
have the benefit of allowing the same API for programs that might run on one machine __or__ across the network
use an ‘address’ that looks like a pathname (‘/tmp/foo.sock’)
What is the default family for the socket we created just a moment ago?

(remember we bound the socket to the symbol foo)

How did you figure this out?

## Socket Types

The socket type determines the semantics of socket communications.

Look up socket type constants with the SOCK_ prefix:

In [23]:
types = get_constants('SOCK_')

In [24]:
types

{<SocketKind.SOCK_STREAM: 1>: 'SOCK_STREAM',
 <SocketKind.SOCK_DGRAM: 2>: 'SOCK_DGRAM',
 <SocketKind.SOCK_RAW: 3>: 'SOCK_RAW',
 <SocketKind.SOCK_RDM: 4>: 'SOCK_RDM',
 <SocketKind.SOCK_SEQPACKET: 5>: 'SOCK_SEQPACKET'}

The most common are 1 (Stream communication (TCP)) and 2 (Datagram communication (UDP)).

What is the default type for our generic socket, foo?

## Socket Protocols

A socket also has a designated protocol. The constants for these are prefixed by IPPROTO_:

In [25]:
protocols = get_constants('IPPROTO_')

In [26]:
protocols

{0: 'IPPROTO_IP',
 1: 'IPPROTO_ICMP',
 6: 'IPPROTO_TCP',
 17: 'IPPROTO_UDP',
 255: 'IPPROTO_RAW'}

The choice of which protocol to use for a socket is determined by the internet layer protocol you intend to use. TCP? UDP? ICMP? IGMP?

What is the default protocol used by our generic socket, foo?

## Customizing Sockets

These three properties of a socket correspond to the three positional arguments you may pass to the socket constructor.

Using them allows you to create sockets with specific communications profiles:

In [27]:
socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

<socket.socket fd=1228, family=AddressFamily.AF_INET, type=SocketKind.SOCK_DGRAM, proto=17>

## Client Side

We’ve already made a socket foo using the generic constructor without any arguments. We can make a better one now by using real address information from a real server online.

In [28]:
streams = [info for info in socket.getaddrinfo('cutecatvideos.net', 'http') if info[1] == socket.SOCK_STREAM]

In [29]:
streams

[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  0,
  '',
  ('50.97.112.130', 80))]

In [30]:
info = streams[0]

In [31]:
client_socket = socket.socket(*info[:3])

## Connecting a Socket

In [32]:
client_socket.connect(info[-1])

* a successful connection returns None
* a failed connection raises an error
* you can use the type of error returned to tell why the connection failed.

## Sending a Message

Send a message to the server on the other end of our connection (we’ll learn in session 2 about the message we are sending):

In [38]:
msg = "GET / HTTP/1.1\r\n"

In [39]:
msg += "Host: 'cutecatvideos.net'\r\n\r\n"

In [40]:
msg = msg.encode('utf8')

In [41]:
msg

b"GET / HTTP/1.1\r\nHost: 'cutecatvideos.net'\r\n\r\n"

In [42]:
client_socket.sendall(msg)

* the transmission continues until all data is sent or an error occurs
* success returns None
* failure to send raises an error
* the type of error can tell you why the transmission failed
* but you cannot know how much, if any, of your data was sent

## Messages are Bytes

One detail from the previous code should stand out:

In [46]:
#msg = msg.encode('utf8')

In [47]:
msg

b"GET / HTTP/1.1\r\nHost: 'cutecatvideos.net'\r\n\r\n"

You can only send bytes through a socket, never unicode

In [48]:
client_socket.sendall(msg.decode('utf8'))

TypeError: a bytes-like object is required, not 'str'

## Receiving a Reply

Whatever reply we get is received by the socket we created. We can read it back out (again, do not type this yet):

In [50]:
response = client_socket.recv(4096)

In [51]:
response[:60]

b'HTTP/1.1 200 OK\r\nDate: Thu, 14 Sep 2017 23:12:22 GMT\r\nServer'

* The sole required argument is buffer_size (an integer). It should be a power of 2 and smallish (~4096)
* It returns a byte string of buffer_size (or smaller if less data was received)
* If the response is longer than buffer size, you can call the method repeatedly. The last bunch will be less than buffer size.

## Cleaning Up

In [52]:
client_socket.close()

## Pulling it all together

First, connect and send a message:

In [53]:
info = socket.getaddrinfo('cutecatvideos.net', 'http')

In [54]:
streams = [i for i in info if i[1] == socket.SOCK_STREAM]

In [55]:
sock_info = streams[0]

In [56]:
msg = "GET / HTTP/1.1\r\n"

In [57]:
msg += "Host: cutecatvideos.net\r\n\r\n"

In [58]:
msg = msg.encode('utf8')

In [59]:
client_socket = socket.socket(*sock_info[:3])

In [60]:
client_socket.connect(sock_info[-1])

In [61]:
client_socket.sendall(msg)

Then, receive a reply, iterating until it is complete:

In [66]:
buffsize = 4096
response = b''
done = False
while not done:
    msg_part = client_socket.recv(buffsize)
    if len(msg_part) < buffsize:
        done = True
        client_socket.close()
        response += msg_part

In [67]:
len(response)

0

## Server Side